# Final assignment Unsupervised Learning

## Background information

This data is from a study of [NSCLC-Radiomics-Genomics](https://wiki.cancerimagingarchive.net/display/Public/NSCLC-Radiomics-Genomics#16056856db10d39adf704eefa53e41edcf5ef41c) and it contains information on 89 patients with non-small cell lung cancer (NSCLC) that were treated with surgery. There are two files, one containing gene expression data and another containing metadata. Both need to be fused before beginning analysis.

# Loading in the data

### Gene Expression data

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pylab
import scipy.stats as stats

In [2]:
df = pd.read_csv('data/GSE58661_series_matrix.txt.gz', compression='gzip', header=61, sep='\t', quotechar='"')
df = df.set_index('ID_REF').transpose().drop('!series_matrix_table_end', axis=1)
df

ID_REF,AFFX-BioB-3_at,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioC-3_at,AFFX-BioC-5_at,AFFX-BioDn-3_at,AFFX-BioDn-5_at,AFFX-BkGr-GC03_at,AFFX-BkGr-GC04_at,AFFX-BkGr-GC05_at,...,merck2-Z41436_at,merck2-Z43503_at,merck2-Z47244_x_at,merck2-Z47250_x_at,merck2-Z48501_s_at,merck2-Z48633_s_at,merck2-Z70222_a_at,merck2-Z70608_x_at,merck2-Z72499_a_at,merck2-Z75331_a_at
GSM1416528,7.376915,6.984530,7.330576,10.922741,11.032030,13.411168,12.625445,1.464847,1.416673,1.438285,...,7.077955,4.350631,7.506299,9.774634,11.295187,10.286077,6.892053,1.777216,9.031084,7.284069
GSM1416529,8.024915,7.427048,8.010530,11.390638,11.533338,13.546671,12.898906,1.502126,1.398866,1.403564,...,7.552333,3.578359,5.516440,4.601167,11.443442,11.173715,5.513603,1.940099,9.527973,6.985508
GSM1416530,7.522543,7.077207,7.334551,10.936703,11.018510,13.360017,12.540647,1.355337,1.343599,1.306212,...,5.084856,2.937384,6.310065,6.315107,11.584115,10.586540,6.112918,1.753519,9.515653,6.556233
GSM1416531,7.152864,6.849513,7.143286,10.791909,10.875259,13.364171,12.579293,1.327548,1.272961,1.354529,...,7.136409,2.904756,5.954062,6.738276,11.240300,10.257621,6.276813,1.889742,9.383670,7.293852
GSM1416532,7.211031,6.753131,7.077163,10.701328,10.823792,13.340075,12.420624,1.296788,1.244117,1.291959,...,7.667150,3.119091,7.257312,8.118139,10.998377,10.634072,7.041829,1.686633,9.464486,6.244336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1416612,7.215856,6.740992,7.152722,10.707980,10.842697,13.332414,12.397369,1.443620,1.342957,1.380656,...,6.044094,3.333540,8.953402,10.221695,11.252406,11.010846,6.493021,1.924313,9.223950,5.817185
GSM1416613,7.465109,7.080787,7.422373,10.736531,10.900509,13.285086,12.356609,1.491890,1.450109,1.436852,...,5.277468,2.861520,6.342812,6.896812,11.694623,9.975013,7.952822,1.783032,9.481911,7.487134
GSM1416614,7.245458,6.725812,7.263596,10.770440,10.838008,13.469373,12.566290,1.441152,1.348752,1.356916,...,7.196013,3.903299,5.724736,5.820700,11.692384,10.044351,6.978308,1.371800,9.632680,7.410402
GSM1416615,7.039592,6.575376,6.975162,10.367009,10.494806,13.214522,12.108750,1.460382,1.362631,1.360318,...,2.952597,2.844055,7.236876,8.625332,10.826930,8.798358,7.460604,1.811532,9.313734,6.396247


In [6]:
print('There seem to be {:.1f} NaNs in the dataframe'.format(df.isna().sum().sum()))

There seem to be 0.0 NaNs in the dataframe


### Meta data

In [7]:
meta = pd.read_excel('data/Lung3.metadata.xls')
meta.head()

,sample.name,title,CEL.file,source.location,organism,characteristics.tag.gender,characteristics.tag.histology,characteristics.tag.tumor.size.maximumdiameter,characteristics.tag.stage.primary.tumor,characteristics.tag.stage.nodes,characteristics.tag.stage.mets,characteristics.tag.primaryVSmets,characteristics.tag.grade,molecule tested,label,platform
0,LUNG3-01,lung_1,LUNG3-01.CEL,Left Lower Lobe,Homo sapiens,M,"Squamous Cell Carcinoma, NOS",4.0,pT2,pN0,pM0,Primary,3,total RNA,biotin,GPL15048
1,LUNG3-02,lung_2,LUNG3-02.CEL,Left Lower Lobe,Homo sapiens,M,"Adenocarcinoma, Papillary, NOS",1.3,pT1,pNX,pMX,Primary,Not Available,total RNA,biotin,GPL15048
2,LUNG3-03,lung_3,LUNG3-03.CEL,Left Lower Lobe,Homo sapiens,M,Non-Small Cell,11.0,pT3,pN0,pM0,Primary,3,total RNA,biotin,GPL15048
3,LUNG3-04,lung_4,LUNG3-04.CEL,Left Lower Lobe,Homo sapiens,M,"Papillary Type AND Adenocarcinoma, Bronch...",NaN,pTX,pNx,pM1,Primary,Not Available,total RNA,biotin,GPL15048
4,LUNG3-05,lung_5,LUNG3-05.CEL,Left Lower Lobe,Homo sapiens,F,"Squamous Cell Carcinoma, NOS",7.8,pT3,pN0,pM0,Primary,2,total RNA,biotin,GPL15048


## Data exploration

### Meta data exploration

In [8]:
meta.shape

(89, 16)

In [9]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 16 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   sample.name                                     89 non-null     object 
 1   title                                           89 non-null     object 
 2   CEL.file                                        89 non-null     object 
 3   source.location                                 89 non-null     object 
 4   organism                                        89 non-null     object 
 5   characteristics.tag.gender                      89 non-null     object 
 6   characteristics.tag.histology                   89 non-null     object 
 7   characteristics.tag.tumor.size.maximumdiameter  88 non-null     float64
 8   characteristics.tag.stage.primary.tumor         89 non-null     object 
 9   characteristics.tag.stage.nodes              

Clearly, most of the meta-data is labels/catergorical data. Features such as organism are redundant considering it was done only to humans and other features also only have a single value/label. For this reason, 'organism', 'characteristics.tag.primaryVSmets', 'molecule tested', 'label' and 'platform' will be dropped. The first three rows also are only used for identification and can be dropped as well.

In [10]:
meta_filtered = meta.drop(['sample.name',
                           'title',
                           'CEL.file',
                           'organism',
                          'characteristics.tag.primaryVSmets',
                          'molecule tested',
                          'label',
                          'platform'], axis=1)

meta_filtered.replace('Not Available', np.nan, inplace=True)
meta_filtered['characteristics.tag.grade'] = meta_filtered['characteristics.tag.grade'].astype('float')

In [11]:
meta_filtered.dtypes

source.location                                    object
characteristics.tag.gender                         object
characteristics.tag.histology                      object
characteristics.tag.tumor.size.maximumdiameter    float64
characteristics.tag.stage.primary.tumor            object
characteristics.tag.stage.nodes                    object
characteristics.tag.stage.mets                     object
characteristics.tag.grade                         float64
dtype: object

The meta-data currently is categorical, a statistical test cannot work with this, for this reason the labels will be mapped to values.

In [12]:
def mapping_dict(df):
    """
    Creates a dict for remapping non-numeric columns.
    """
    
    col_dict = {}
    for col in df.columns:
        if df[col].dtypes != 'object':
            pass
        else:
            u_vals = df[col].unique()
            count = -1.0
            unique_dict = {}
            for val in u_vals:
                count += 1.0
                unique_dict[val] = count

            col_dict[col] = unique_dict

    return col_dict

def df_remapper(df, map_dict):
    """
    Uses a dictionairy to remap all non-numeric columns
    """
    for col in df.columns:
        if df[col].dtypes != 'object':
            pass
        else:
            df[col] = df[col].map(map_dict[col])
    return df


mapdict = mapping_dict(meta_filtered)
df_mapped = df_remapper(meta_filtered, mapdict)
df_mapped

,source.location,characteristics.tag.gender,characteristics.tag.histology,characteristics.tag.tumor.size.maximumdiameter,characteristics.tag.stage.primary.tumor,characteristics.tag.stage.nodes,characteristics.tag.stage.mets,characteristics.tag.grade
0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0
1,0.0,0.0,1.0,1.3,1.0,1.0,1.0,NaN
2,0.0,0.0,2.0,11.0,2.0,0.0,0.0,3.0
3,0.0,0.0,3.0,NaN,3.0,2.0,2.0,NaN
4,0.0,1.0,0.0,7.8,2.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...
84,3.0,0.0,8.0,2.2,10.0,0.0,0.0,NaN
85,3.0,0.0,0.0,5.0,2.0,5.0,0.0,3.0
86,4.0,0.0,15.0,3.5,0.0,0.0,0.0,2.0
87,3.0,0.0,16.0,8.5,2.0,0.0,0.0,NaN


### Gene Expression Data

In [13]:
df.shape

(89, 60607)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89 entries, GSM1416528 to GSM1416616
Columns: 60607 entries, AFFX-BioB-3_at to merck2-Z75331_a_at
dtypes: float64(60607)
memory usage: 41.2+ MB


There seem to be 89 rows and 60607 columns. This definitly is a very large number of dimensions and needs to be scaled down.

A correlation matrix will be made to investigate correlation within the dataset. Highly correlated features can be removed as these only bring redundant information. This is preferably removed to create a simpler model with less features.

In [16]:
# corr_matrix = df.corr(
#     method = 'pearson',  # Correlation method
#     min_periods = 1      # Min number of observations required
# )

# # set diagonal to 0.0 to remove self-correlation
# np.fill_diagonal(corr_matrix.values, 0.0)

# corr_matrix

MemoryError: Unable to allocate 27.4 GiB for an array with shape (60607, 60607) and data type float64

In [ ]:
# Visualize the most highlt correlated columns
corr_matrix.abs().idxmax()

In [ ]:
# Check how hight the correlation is.
corr_matrix.abs().max()

In [ ]:
# .skew 0: no skew, + right skew, - left skew, look for above .75 
skew_columns = (df_unlabel.skew().sort_values(ascending=False))
print('-- skew --')
print(skew_columns)
print('\n')

skew_columns_75 = skew_columns.loc[skew_columns > 0.75]
print('-- skew > 0.75 --')
print(skew_columns_75)

print('-- max skew --')
print(skew_columns.max())
print('-- min skew --')
print(skew_columns.min())

In [ ]:
# Plot a histogram to view the data distribution
plt.subplots(figsize =(12, 12))
for col in df_unlabel.columns:
    plt.hist(df_unlabel[col], bins='auto')
plt.show()

According to the above information, the dataset initially seemed to have a slight skew (skew in this dataset varies from 0.39 and 0.58) and after verifying it with a histogram, it clearly seems to be skewed.

In [ ]:
for col in df_unlabel.columns:
    stats.probplot(df_unlabel[col], dist="norm", plot=pylab)
pylab.show()

## Data pre-processing

To resolve the data skewness a number of techniques can be employed, the aim is to get a number as close as possible to zero, such there is minimal skewing in the data.

In [ ]:
# log transform
df_log = np.log(df_unlabel)
print('Log transform skew: min: {:.3f}, max: {:.3f}'.format(df_log.skew().min(), df_log.skew().max()))
df_log.head()

In [ ]:
# square root transform
df_sqrt = np.sqrt(df_unlabel)
print('Square root transform skew: min: {:.3f}, max: {:.3f}'.format(df_sqrt.skew().min(), df_sqrt.skew().max()))
df_sqrt.head()

In [ ]:
# Plot a histogram to view the data distribution
plt.subplots(figsize =(12, 12))
for col in df_sqrt.columns:
    plt.hist(df_sqrt[col], bins='auto')
plt.show()

In [ ]:
for col in df_sqrt.columns:
    stats.probplot(df_sqrt[col], dist="norm", plot=pylab)
pylab.show()